In [1]:
# Initialization
from pds4_tools import pds4_read       # to read and inspect the data and metadata
import matplotlib.pyplot as plt        # for plotting
import matplotlib
import numpy as np
import pandas as pd
from PIL import Image

import cv2 as cv # HDR tonemapping trial

from skimage import exposure
from skimage import data, img_as_float
import colour
from colour.plotting import *
import glob, os # for batch reading, processing and exporting in directories

from colour_demosaicing import (
    demosaicing_CFA_Bayer_bilinear,
    demosaicing_CFA_Bayer_Malvar2004,
    demosaicing_CFA_Bayer_Menon2007,
    mosaicing_CFA_Bayer)

cctf_encoding = colour.cctf_encoding
_ = colour.utilities.filter_warnings()
# colour.utilities.describe_environment();

def read_pds(path, payload='NaTeCam'):
    data = pds4_read(path, quiet=True)
    img = np.array(data[0].data)
    img = img_as_float(img)
    if payload == 'MoRIC':
        coor = pd.DataFrame(data[1].data)
    return img, coor

def debayer_img(img, CFArr='RGGB'):
    debayered = cctf_encoding(demosaicing_CFA_Bayer_bilinear(img, CFArr))
    return debayered

def stretch_img(img, percent=0.5):
    # cf https://www.harrisgeospatial.com/docs/BackgroundStretchTypes.html
    # Adapts for different percentages
    p2, p98 = np.percentile(img, (0+percent, 100-percent))
    img = exposure.rescale_intensity(img, in_range=(p2, p98))
    return img

def check_type(p): # Check PCAM image type
    ty = p.split('_')[2].split('-')[1]
    return ty

def plot_img_and_hist(image, ty='Q', size=10, hist=True, bins=128): # ty indicates the type of the data product
    """Plot an image along with its histogram
    """
    if hist:
        fig, axes = plt.subplots(2,1, figsize=(size,size), gridspec_kw={'height_ratios': [3, 1]})
        ax_img, ax_hist = axes
    else:
        fig, ax_img = plt.subplots(figsize=(size,size))

    # Display image
    ax_img.imshow(image)
    # Without axes on image
    ax_img.set_axis_off()

    if hist:
        # Display histogram
        if ty == 'Q': # Panchromatic
          ax_hist.hist(image.ravel(), bins=bins, histtype='step', color='black')
        elif ty == 'C': # Colour
          ax_hist.hist(image[:,:,0].ravel(), bins=bins, histtype='step', color='red')
          ax_hist.hist(image[:,:,1].ravel(), bins=bins, histtype='step', color='green')
          ax_hist.hist(image[:,:,2].ravel(), bins=bins, histtype='step', color='blue')
        else: # Unknown type
          print('Image data type not recognized')
          pass

        ax_hist.ticklabel_format(axis='y', style='scientific', scilimits=(0, 0))
        ax_hist.set_xlabel('Pixel intensity')
        ax_hist.set_xlim(0, 1)
        ax_hist.set_yticks([])
    
def export_img(name, img):
    pil_img = Image.fromarray(np.clip(img*255, 0, 255).astype('uint8'))
    pil_img.save(name)

def preview_save(img, ty, size=10, hist=False):
    print('Generating preview...')
    plot_img_and_hist(img, ty, size, hist)
    plt.show()
    print('Saving image...')
    export_img('{}.png'.format(p), img)

def prune_coor(coor):
    size_x = coor.iloc[-1]['Column']
    size_y = coor.iloc[-1]['Row']

    upper_edge = coor[coor['Row'] == 1]
    lower_edge = coor[coor['Row'] == size_y]
    left_edge = coor[coor['Column'] == 1]
    right_edge = coor[coor['Column'] == size_x]

    return pd.concat([upper_edge, lower_edge, left_edge, right_edge]).drop_duplicates().reset_index(drop=True)

def preview_img_location(img, coor, coor_map, size=5):
    pruned_coor = prune_coor(coor)
    fig = plt.figure(figsize=(size, size), constrained_layout=True)
    widths = [1.75, 1]
    heights = [1]
    spec = fig.add_gridspec(nrows=1, ncols=2, width_ratios=widths, height_ratios=heights)
    ax_img = fig.add_subplot(spec[0, 0])
    ax_map = fig.add_subplot(spec[0, 1])
    ax_img.imshow(img)
    ax_img.set_axis_off()
    ax_map.imshow(coor_map, extent = [-180, 180, -90, 90], aspect=1)
    ax_map.plot(pruned_coor['Longitude'], pruned_coor['Latitude'], '.y', alpha=1, markersize=1.5)

In [2]:
# Load File List
directory = r'C:\Users\Darren Wu\Desktop\SpaceInfos\2023\TW1Cont\MoRIC\Batch'
directory = directory.replace(os.sep, '/')
os.chdir(directory)
p = []
coor_map = plt.imread('Mars_Viking_ClrMosaic_global_925m-2.jpg')
for filename in glob.glob('*.2CL'):
    if ('Preview_' + filename.removesuffix('.2CL') + '.png') not in glob.glob('*.png'):
        p.append(filename)
print('File list loaded.')
#print(p)

File list loaded.


In [3]:
for filename in p:
    img, coor = read_pds(filename, payload='MoRIC')
    print('Processing and exporting file ({index}) {name}'.format(index = p.index(filename)+1, name = filename.removesuffix('.2CL')))
    
    # Preview image
    # ty = 'C'
    # print(img)

    #img = debayer_img(img)
    #img = img.astype('float32')
    #tonemap1 = cv.createTonemap(gamma=1)
    #tonemap1 = cv.createTonemapMantiuk(gamma=2, scale=1.1, saturation=1.6)
    #tonemap1 = cv.createTonemapReinhard(gamma=0.5, intensity=-2, light_adapt=1, color_adapt=0.5)
    #tonemap1 = cv.createTonemapDrago(gamma=1, saturation=0.8, bias=0.8)
    #img = tonemap1.process(img)
        
    #img = stretch_img(img)

    print('Generating previews...')
    preview_img_location(img, coor, coor_map, size=20)
    plt.savefig('Preview_{}.png'.format(filename.removesuffix('.2CL')), bbox_inches='tight', facecolor='white')
    #plt.show()
    plt.close()

    print('DONE')
    

Processing and exporting file (1) HX1-Or_GRAS_MoRIC-F-0046_SCI_N_20211122003415_20211122003415_00602_A
Generating previews...
DONE
Processing and exporting file (2) HX1-Or_GRAS_MoRIC-F-0046_SCI_N_20211122214817_20211122214817_00605_A
Generating previews...
DONE
Processing and exporting file (3) HX1-Or_GRAS_MoRIC-F-0046_SCI_N_20211123190212_20211123190212_00608_A
Generating previews...
DONE
Processing and exporting file (4) HX1-Or_GRAS_MoRIC-F-0046_SCI_N_20211124161609_20211124161609_00611_A
Generating previews...
DONE
Processing and exporting file (5) HX1-Or_GRAS_MoRIC-F-0046_SCI_N_20211126104402_20211126104402_00617_A
Generating previews...
DONE
Processing and exporting file (6) HX1-Or_GRAS_MoRIC-F-0046_SCI_N_20211127075800_20211127075800_00620_A
Generating previews...
DONE
Processing and exporting file (7) HX1-Or_GRAS_MoRIC-F-0046_SCI_N_20211128051153_20211128051153_00623_A
Generating previews...
DONE
Processing and exporting file (8) HX1-Or_GRAS_MoRIC-F-0046_SCI_N_20211129233951_202